In [1]:
from holoclean.holoclean import HoloClean, Session

holo =      HoloClean(
            holoclean_path="..",         # path to holoclean package
            verbose=True,
            # to limit possible values for training data
            pruning_threshold1=0.1,
            # to limit possible values for training data to less than k values
            pruning_clean_breakoff=6,
            # to limit possible values for dirty data (applied after
            # Threshold 1)
            pruning_threshold2=0,
            # to limit possible values for dirty data to less than k values
            pruning_dk_breakoff=6,
            # learning parameters
            learning_iterations=30,
            learning_rate=0.001,
            batch_size=5
        )
session = Session(holo)

/Users/joshmcgrath/anaconda2/envs/torch/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
data_path = "food_input_transf.csv"
data = session.load_data(data_path)

Time to Load Data: 34.9791038036



In [3]:
data.show(5)

+--------------------+------------+-------+-----+-------+------------+-----------+--------------+---------------+-------------------+-------+------------+--------------------+--------------------+--------------------+-----+-----+
|             akaname|inspectionid|   city|state|results|   longitude|   latitude|inspectiondate|           risk|           location|license|facilitytype|             address|      inspectiontype|             dbaname|  zip|__ind|
+--------------------+------------+-------+-----+-------+------------+-----------+--------------+---------------+-------------------+-------+------------+--------------------+--------------------+--------------------+-----+-----+
|  sunrise gyros inc.|     1967112|chicago|   il|   pass|-87.65378117|41.74659471|      20161024|risk 2 (medium)|(41.746594707551054|1959206|  restaurant|   8106 s racine ave|           complaint|  sunrise gyros inc.|60620|    1|
|taqueria los coma...|     1967089|chicago|   il|   pass|-87.72273813| 41.783097

In [4]:
data.printSchema()

root
 |-- akaname: string (nullable = true)
 |-- inspectionid: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- results: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- inspectiondate: string (nullable = true)
 |-- risk: string (nullable = true)
 |-- location: string (nullable = true)
 |-- license: string (nullable = true)
 |-- facilitytype: string (nullable = true)
 |-- address: string (nullable = true)
 |-- inspectiontype: string (nullable = true)
 |-- dbaname: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- __ind: long (nullable = true)



## load DCs

In [5]:
dc_path = "food_constraints.txt"
dcs = session.load_denial_constraints(dc_path)


In [6]:
dcs

['t1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&IQ(t1.facilitytype,t2.facilitytype)',
 't1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&IQ(t1.address,t2.address)',
 't1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&IQ(t1.city,t2.city)',
 't1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&IQ(t1.state,t2.state)',
 't1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&IQ(t1.zip,t2.zip)',
 't1&t2&EQ(t1.zip,t2.zip)&IQ(t1.state,t2.state)',
 "t1&t2&EQ(t1.dbaname,t2.dbaname)&EQ(t1.location,t2.location)&EQ(t1.results,'Out of Business')&LTE(t1.inspectiondate,t2.inspectiondate)"]

In [7]:
from holoclean.errordetection.sql_dcerrordetector import SqlDCErrorDetection

# instantiate Holoclean's built in error detector
detector = SqlDCErrorDetection(session)

# both clean and dirty sets are returned as pyspark dataframes
error_detector_list =[]
error_detector_list.append(detector)
clean, dirty = session.detect_errors(error_detector_list)

Time for Error Detection: 170.763827801



In [8]:
clean.show(5)
dirty.show(5)

+---+------------+
|ind|        attr|
+---+------------+
|  8|inspectionid|
| 32|facilitytype|
| 40|     dbaname|
| 69|     license|
| 78|       state|
+---+------------+
only showing top 5 rows

+------+--------+
|   ind|    attr|
+------+--------+
| 91507| dbaname|
| 91489|    city|
|111890|location|
| 26602|location|
|103701| dbaname|
+------+--------+
only showing top 5 rows



In [9]:
repaired = session.repair()

Time for Domain Pruning: 143.605797052

Time for Featurization: 372.22617507



  0%|          | 0/30 [00:00<?, ?it/s]/Users/joshmcgrath/git/holo-george/holoclean/learning/softmax.py:224: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return output.data[0]
  3%|▎         | 1/30 [00:18<08:50, 18.31s/it]

Epoch 1, cost = 0.923236, acc = 98.82%


  7%|▋         | 2/30 [00:37<08:40, 18.59s/it]

Epoch 2, cost = 0.925345, acc = 98.82%


...

Epoch 29, cost = 0.925345, acc = 98.82%


100%|██████████| 30/30 [08:08<00:00, 16.30s/it]

Epoch 30, cost = 0.925345, acc = 98.82%


Time for Training Model: 506.181448936

Time for Test Featurization: 0.181910991669

Time for Inference: 20.8748679161



/Users/joshmcgrath/git/holo-george/holoclean/learning/softmax.py:420: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  msg = "Feature " + str(count) + ": " + str(weight[0].data[0])


In [10]:
session.compare_to_truth("../food_data/food_clean_transform.csv")

We have detected 356 errors out of 366 total
The top-1 precision  is : 0.761
The top-1 recall is : 0.643 over the 356 errors found during error detection


In [14]:
s

+-----+--------------------+------------+-------+-----+-------+------------+-----------+--------------+---------------+-------------------+-------+------------+--------------------+--------------------+--------------------+-----+
|index|             akaname|inspectionid|   City|State|Results|   longitude|   latitude|InspectionDate|           risk|           Location|license|FacilityType|             Address|      inspectiontype|             DBAName|  Zip|
+-----+--------------------+------------+-------+-----+-------+------------+-----------+--------------+---------------+-------------------+-------+------------+--------------------+--------------------+--------------------+-----+
|    1|  sunrise gyros inc.|     1967112|chicago|   il|   pass|-87.65378117|41.74659471|      20161024|risk 2 (medium)|(41.746594707551054|1959206|  restaurant|   8106 s racine ave|           complaint|  sunrise gyros inc.|60620|
|    2|taqueria los coma...|     1967089|chicago|   il|   pass|-87.72273813| 41.

In [ ]:
session.holo_env.dataengine.db_backend[1].rollback()